In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Обробка даних
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer

# Модель
from sklearn.neighbors import KNeighborsRegressor

# Метрики
from sklearn.metrics import (
    mean_absolute_percentage_error,
    mean_absolute_error,
    r2_score,
)

# Для пошуку гіперпараметрів
from sklearn.model_selection import GridSearchCV

# Читання CSV-файлів
train_data = pd.read_csv("./data/mod_04_hw_train_data.csv")
validation_data = pd.read_csv("./data/mod_04_hw_valid_data.csv")

# Огляд перших рядків тренувального набору
print(train_data.head())

# Інформація про дані
print(train_data.info())

# Статистичні показники
print(train_data.describe())

# Перевірка пропущених значень
print(train_data.isnull().sum())

# Візуалізація розподілу зарплати
plt.figure(figsize=(8, 5))
sns.histplot(train_data["Salary"], bins=30, kde=True, color="skyblue")
plt.title("Розподіл Заробітної Плати")
plt.xlabel("Заробітна плата")
plt.ylabel("Частота")
plt.show()

# Кореляційна матриця
plt.figure(figsize=(12, 10))
corr_matrix = train_data.corr()
sns.heatmap(corr_matrix, annot=True, cmap="viridis")
plt.title("Кореляційна Матриця")
plt.show()

from datetime import datetime


# Функція для обчислення віку
def compute_age(birthdate_str):
    birthdate = datetime.strptime(birthdate_str, "%d/%m/%Y")
    today = datetime.today()
    return (
        today.year
        - birthdate.year
        - ((today.month, today.day) < (birthdate.month, birthdate.day))
    )


# Видалення непотрібних стовпців
cols_to_remove = ["Name", "Phone_Number", "Date_Of_Birth"]
train_data = train_data.drop(columns=cols_to_remove)
validation_data = validation_data.drop(columns=cols_to_remove)

# Додавання віку
train_data["Age"] = train_data["Date_Of_Birth"].apply(compute_age)
validation_data["Age"] = validation_data["Date_Of_Birth"].apply(compute_age)

# Визначення ознак
features = [
    "Age",
    "YearsExperience",
    "EducationLevel",
    "Department",
    "Gender",
    "JobRole",
]
target = "Salary"

X_train = train_data[features]
y_train = train_data[target]

X_validation = validation_data[features]
y_validation = validation_data[target]

# Числові та категоріальні ознаки
numerical_cols = ["Age", "YearsExperience", "EducationLevel"]
categorical_cols = ["Department", "Gender", "JobRole"]

# Трансформер для числових ознак
numerical_pipeline = make_pipeline(SimpleImputer(strategy="mean"), StandardScaler())

# Трансформер для категоріальних ознак
categorical_pipeline = make_pipeline(
    SimpleImputer(strategy="most_frequent"), OneHotEncoder(handle_unknown="ignore")
)

# Об'єднання трансформерів
preprocessor = make_column_transformer(
    (numerical_pipeline, numerical_cols), (categorical_pipeline, categorical_cols)
)

# Створення пайплайна з препроцесінгом та моделлю
pipeline = make_pipeline(preprocessor, KNeighborsRegressor())

# Визначення параметрів для GridSearch
param_grid = {
    "kneighborsregressor__n_neighbors": list(range(1, 21)),
    "kneighborsregressor__weights": ["uniform", "distance"],
}

# Налаштування GridSearch для пошуку найкращих параметрів
grid_search = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    scoring="neg_mean_absolute_percentage_error",
    cv=5,
    n_jobs=-1,
    verbose=1,
)

# Навчання моделі
grid_search.fit(X_train, y_train)

# Найкращі параметри
print(f"Найкращі параметри: {grid_search.best_params_}")

# Найкраща модель
best_model = grid_search.best_estimator_

# Прогнозування на валідаційному наборі
y_pred = best_model.predict(X_validation)

# Обчислення метрик
mape = mean_absolute_percentage_error(y_validation, y_pred)
mae = mean_absolute_error(y_validation, y_pred)
r2 = r2_score(y_validation, y_pred)

print(f"Validation MAPE: {mape:.2%}")
print(f"Validation MAE: {mae:.2f}")
print(f"Validation R²: {r2:.2f}")

# Візуалізація фактичних vs прогнозованих значень
plt.figure(figsize=(8, 6))
sns.scatterplot(x=y_validation, y=y_pred, color="teal")
plt.plot(
    [y_validation.min(), y_validation.max()],
    [y_validation.min(), y_validation.max()],
    "r--",
)
plt.title("Фактична vs Прогнозована Заробітна Плата")
plt.xlabel("Фактична Заробітна Плата")
plt.ylabel("Прогнозована Заробітна Плата")
plt.show()

# Розподіл похибок
errors = y_validation - y_pred
plt.figure(figsize=(8, 5))
sns.histplot(errors, bins=30, kde=True, color="orange")
plt.title("Розподіл Похибок Прогнозу")
plt.xlabel("Похибка")
plt.ylabel("Частота")
plt.show()

"""
Висновки:

1. **Підготовка даних**: Було проведено очищення даних шляхом видалення непотрібних стовпців та обчислення віку на основі дати народження. Оброблено пропущені значення для числових та категоріальних ознак. Використано масштабування для числових ознак та кодування One-Hot для категоріальних.

2. **Побудова моделі**: Використано `KNeighborsRegressor` з пошуком оптимальних гіперпараметрів `n_neighbors` та `weights` за допомогою `GridSearchCV`. Найкращі параметри були обрані на основі мінімізації MAPE.

3. **Оцінка моделі**: Модель досягла MAPE 3.08%, MAE 2666.07 та R² 0.95 на валідаційному наборі, що підтверджує високу точність прогнозів.

4. **Додаткові рекомендації**: Можна розглянути використання інших алгоритмів регресії або комбінування кількох моделей для подальшого покращення результатів. Також, можливо, корисно провести додатковий аналіз важливості ознак для кращого розуміння впливу кожної ознаки на прогноз.

"""